In [3]:
import torch
import torch.nn as NeuralNet
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
num_epochs = 10
batch_size = 4
learning_rate = 0.001

transform = transforms.Compose([transforms.ToTensor(), 
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

training_dataset = torchvision.datasets.CIFAR10('Data', train=True, download=True,
                                                 transform=transform)
testing_dataset = torchvision.datasets.CIFAR10('Data', train=False, transform=transform)

training_loader = torch.utils.data.DataLoader(dataset=training_dataset,
                                                batch_size=batch_size, shuffle=True)
testing_loader = torch.utils.data.DataLoader(dataset=testing_dataset,
                                                batch_size=batch_size, shuffle=False)
                                    
classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



Files already downloaded and verified


In [24]:
class ConvNet(NeuralNet.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = NeuralNet.Conv2d(in_channels=3, out_channels=6, kernel_size=5)

        self.pool= NeuralNet.MaxPool2d(kernel_size=2, stride=2)

        self.Conv2 = NeuralNet.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        
        self.dense1 = NeuralNet.Linear(in_features=16*5*5, out_features=120)

        self.dense2 = NeuralNet.Linear(in_features=120, out_features=84)

        self.dense3 = NeuralNet.Linear(in_features=84, out_features=10) 

    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.Conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16*5*5)
        x =  F.relu(self.dense1(x))
        x =  F.relu(self.dense2(x))
        x =  self.dense3(x)
        return x

In [25]:
model= ConvNet().to(device)

In [26]:
# loss and optimiser

criterion = NeuralNet.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [27]:
n_total_steps = len(training_loader)

for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(training_loader):

        images = images.to(device)
        labels = labels.to(device)

        ouputs = model(images)

        loss = criterion(ouputs, labels)

        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()


        if (i+1) % 2000 == 0:
            print(f'epoch {epoch + 1} / {num_epochs}, step {i + 1} / {n_total_steps}, loss = {loss.item():.4f}')

print('training finished')

epoch 1 / 10, step 2000 / 12500, loss = 2.3168
epoch 1 / 10, step 4000 / 12500, loss = 2.3104
epoch 1 / 10, step 6000 / 12500, loss = 2.3142
epoch 1 / 10, step 8000 / 12500, loss = 2.2715
epoch 1 / 10, step 10000 / 12500, loss = 2.3898
epoch 1 / 10, step 12000 / 12500, loss = 1.8506
epoch 2 / 10, step 2000 / 12500, loss = 2.0526
epoch 2 / 10, step 4000 / 12500, loss = 2.0624
epoch 2 / 10, step 6000 / 12500, loss = 2.2503
epoch 2 / 10, step 8000 / 12500, loss = 2.9184
epoch 2 / 10, step 10000 / 12500, loss = 1.5642
epoch 2 / 10, step 12000 / 12500, loss = 1.9651
epoch 3 / 10, step 2000 / 12500, loss = 2.0121
epoch 3 / 10, step 4000 / 12500, loss = 1.6561
epoch 3 / 10, step 6000 / 12500, loss = 1.2073
epoch 3 / 10, step 8000 / 12500, loss = 1.6669
epoch 3 / 10, step 10000 / 12500, loss = 1.5073
epoch 3 / 10, step 12000 / 12500, loss = 1.0615
epoch 4 / 10, step 2000 / 12500, loss = 2.0125
epoch 4 / 10, step 4000 / 12500, loss = 1.0413
epoch 4 / 10, step 6000 / 12500, loss = 1.2179
epoch 4

In [28]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in testing_loader:
        images = images.to(device)
        labels = labels.to(device)
        ouputs = model(images)

        _, Prediction = torch.max(ouputs, 1)
        n_samples += labels.size(0)
        n_correct = (Prediction == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = Prediction[i]
            if(label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    
    acc = 100 * n_correct / n_samples

    print(f'accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

accuracy of the network: 0.03 %
Accuracy of plane: 52.0 %
Accuracy of car: 55.2 %
Accuracy of bird: 38.4 %
Accuracy of cat: 25.5 %
Accuracy of deer: 39.0 %
Accuracy of dog: 61.6 %
Accuracy of frog: 75.2 %
Accuracy of horse: 60.5 %
Accuracy of ship: 71.8 %
Accuracy of truck: 69.9 %
